In [73]:
import pandas as pd 
import numpy as np 
import regex
import re 
import unicodedata as ud

# Exercice qualité de l'air

In [74]:
qualite = pd.read_csv('openaq1.csv', sep=';')
qualite

,Country Code,City,Location,Coordinates,Pollutant,Source Name,Unit,Value,Last Updated,Country Label
0,DE,Manfred Lotz,"Cottbus, W.-Külz-Str.","51.75413,14.325549",PM2.5,EEA Germany,µg/m³,21.7,2016-12-21T12:00:00+01:00,Germany
1,ES,Castellón/Castelló,ES1969A,"39.8236111095,-0.24444444",NO2,EEA Spain,µg/m³,1.0,2021-07-13T08:00:00+02:00,Spain
2,ES,Valencia/València,ES1885A,"39.4802777695,-0.33638888",SO2,EEA Spain,µg/m³,5.0,2021-08-06T11:00:00+02:00,Spain
3,ES,Alicante/Alacant,ES2008A,"37.9911111095,-0.69",PM2.5,EEA Spain,µg/m³,9.0,2021-08-06T11:00:00+02:00,Spain
4,ES,Alicante/Alacant,ES1968A,"38.3511111095,-0.51388888",PM10,EEA Spain,µg/m³,1.0,2021-08-06T11:00:00+02:00,Spain
...,...,...,...,...,...,...,...,...,...,...
40446,US,BERNALILLO,WESTSIDE - 9 Mile,"35.064098,-106.761292",PM10,AirNow,µg/m³,10.0,2020-07-17T14:00:00+02:00,United States
40447,US,Asheville,Skyland DRR,"35.481861,-82.509861",SO2,AirNow,ppm,0.0,2020-07-02T12:00:00+02:00,United States
40448,US,San Antonio,San Antonio IH35 C10,"29.5294,-98.39139",PM2.5,AirNow,µg/m³,6.0,2020-06-26T01:00:00+02:00,United States
40449,US,Cleveland-Elyria-Mentor,Cleveland Near Road,"41.4409,-81.4949",PM2.5,AirNow,µg/m³,15.3,2020-05-26T16:00:00+02:00,United States


## Supprimer les données qui ne concernent pas la journée du 06-08-2021

In [75]:
#qualite[['Last Updated','Hour']] = qualite['Last Updated'].str.split('T',expand=True)
qualite = qualite[qualite['Last Updated'].str.contains("2021-08-06")]

qualite

,Country Code,City,Location,Coordinates,Pollutant,Source Name,Unit,Value,Last Updated,Country Label
2,ES,Valencia/València,ES1885A,"39.4802777695,-0.33638888",SO2,EEA Spain,µg/m³,5.000,2021-08-06T11:00:00+02:00,Spain
3,ES,Alicante/Alacant,ES2008A,"37.9911111095,-0.69",PM2.5,EEA Spain,µg/m³,9.000,2021-08-06T11:00:00+02:00,Spain
4,ES,Alicante/Alacant,ES1968A,"38.3511111095,-0.51388888",PM10,EEA Spain,µg/m³,1.000,2021-08-06T11:00:00+02:00,Spain
5,ES,Valencia/València,ES1911A,"39.7052777695,-0.33666666",CO,EEA Spain,µg/m³,100.000,2021-08-06T11:00:00+02:00,Spain
6,ES,Valencia/València,ES1185A,"39.6672222195,-0.23472222",PM10,EEA Spain,µg/m³,13.000,2021-08-06T11:00:00+02:00,Spain
...,...,...,...,...,...,...,...,...,...,...
40408,TH,Bangkok,"Din Dang, Bangkok","13.7619223,100.558606",PM10,Air4Thai,µg/m³,42.000,2021-08-06T14:00:00+02:00,Thailand
40409,TH,Samut Prakan,"City Hall, Samut Prakan","13.599149,100.597345",SO2,Air4Thai,ppm,0.000,2021-08-06T13:00:00+02:00,Thailand
40410,TH,Samut Prakan,"City Hall, Samut Prakan","13.599149,100.597345",NO2,Air4Thai,ppm,0.006,2021-08-06T13:00:00+02:00,Thailand
40411,TH,Chachoengsao,Municipality Office Tungsadao,"13.588554,101.286359",PM10,Air4Thai,µg/m³,37.000,2021-08-06T13:00:00+02:00,Thailand


## Supprimer les données qui sont illisibles et pour lesquelles on ne comprend pas la ville concernée

In [76]:
#qualite = qualite[qualite['City'].str.contains(encoding ="utf_8")]

In [77]:
#def lettre(text):
    stripped_text = ''
    for c in text:
        stripped_text += c if len(c.encode(encoding='utf_8'))==1 else ''
        return(stripped_text)
#qualite['City'] = qualite['City'].apply(lettre)

IndentationError: unexpected indent (Temp/ipykernel_10076/3953921352.py, line 2)

## Pour chaque polluant, vérifier qu'on n'a qu'une seule mesure utilisée. Autrement, supprimer les données qui correspondent à la mesure minoritaire ou moins cohérente.

In [95]:
qualite = qualite[qualite['City'].str.contains("A")]

ValueError: Cannot mask with non-boolean array containing NA / NaN values

## Regarder s'il y a des valeurs aberrantes ou des problèmes par rapport à notre objectif final (pouvoir comparer des mesures sur un maximum de pays) et noter ces aspects dans un fichier texte.

In [82]:
latin_letters= {}

def is_latin(uchr):
    try: return latin_letters[uchr]
    except KeyError:
         return latin_letters.setdefault(uchr, 'LATIN' in ud.name(uchr))

def only_roman_chars(unistr):
    return all(is_latin(uchr)
           for uchr in unistr
           if uchr.isalpha()) # isalpha suggested by John Machin






In [87]:
qualite['City'] = qualite['City'].apply(only_roman_chars)

TypeError: 'float' object is not iterable

In [97]:
qualite.to_csv('qualite2qualite.csv' , index=False)